In [2]:
import pandas as pd
import numpy as np
import pingouin as pg
from scipy import stats
from collections import Counter
pd.options.display.max_columns = None
pd.options.display.max_rows = None
pd.options.display.max_colwidth = 100
#df = pd.read_csv('/Users/louk/Desktop/Master Thesis/Experiment Files/Product Description Rating_TestData.csv')
df = pd.read_csv('/Users/louk/Desktop/Master Thesis/Experiment Files/FinalProductDescriptionRatingFile.csv')

In [3]:
df = df.loc[[val for val in df.index if val != 17]]
alist = [elem for sublist in [[0], list(range(2, 25)), [28]] for elem in sublist]
df = df.loc[[val for val in alist if val != 17]]

In [27]:
#question numbers of the haptic

In [28]:
one_shot_readability = df[['Q109', 'Q143', 'Q45', 'Q149']].loc[2:]
few_shot_readability = df[['Q117', 'Q22', 'Q53', 'Q77']].loc[2:]
fine_tune_readability = df[['Q125', 'Q29', 'Q61', 'Q69']].loc[2:]
###
one_shot_correctness = df[['Q111', 'Q144', 'Q47', 'Q150']].loc[2:]
few_shot_correctness = df[['Q119', 'Q24', 'Q55', 'Q79']].loc[2:]
fine_tune_correctness = df[['Q127', 'Q31', 'Q63', 'Q71']].loc[2:]
###
one_shot_repetition = df[['Q113', 'Q145', 'Q49', 'Q151']].loc[2:]
few_shot_repetition = df[['Q121', 'Q26', 'Q57', 'Q81']].loc[2:]
fine_tune_repetition = df[['Q129', 'Q33', 'Q65', 'Q73']].loc[2:]

In [29]:
def return_cronbach_alpha_touch_metrics(df):
    Touch_1 = ['Q74', 'Q82', 'Q152', 'Q66', 'Q58', 'Q50', 'Q34', 'Q27', 'Q146', 'Q130', 'Q122', 'Q114']
    Touch_2 = ['Q75', 'Q83', 'Q153', 'Q67', 'Q59', 'Q51', 'Q35', 'Q28', 'Q147', 'Q131', 'Q123', 'Q115'] 
    touch_1df = df[Touch_1][2:]
    touch_2df = df[Touch_2][2:]
    
    touch_1_list = []
    for column in touch_1df.columns:
        touch_1_list.append([elem for elem in touch_1df[column].tolist() if type(elem) != float])

    touch_1_list_int = [int(a) for b in touch_1_list for a in b]

    touch_2_list = []
    for column in touch_2df.columns:
        touch_2_list.append([elem for elem in touch_2df[column].tolist() if type(elem) != float])

    touch_2_list_int = [int(a) for b in touch_2_list for a in b]
    
    stackedDFtouch1 = touch_1df.stack(dropna=False).reset_index()
    stackedDFtouch2 = touch_2df.stack(dropna=False).reset_index()
    stackedDFtouch2 = stackedDFtouch2.rename(columns={0:'score'})
    
    result = pd.concat([stackedDFtouch1, stackedDFtouch2], axis=1)
    result[0] = np.float32(result[0])
    result['score'] = np.float32(result['score'])
    
    return pg.cronbach_alpha(data=result[[0, 'score']])

In [30]:
return_cronbach_alpha_touch_metrics(df)

(0.8301736007991576, array([0.784, 0.867]))

In [43]:
def t_test_instruction_types(df1, df2, df3, score = ['correctness', 'repitition', 'readability']):
    names =['oneshot', 'fewshot', 'finetune']
    dflist = [df1, df2, df3]

    comparisondict = {'df1':dict(), 'df2':dict(), 'df3':dict()}
    for dfname, df in list(zip(comparisondict.keys(), dflist)):
        for column in df.columns:
            comparisondict[dfname][column] = [int(a) for b in [elem for elem in df[column].tolist() if type(elem) != float] for a in b]
    
    for key in comparisondict.keys():
        subdict = comparisondict[key]
        comparisondict[key] = sum(subdict.values(), [])
    
    dfnameslist = list(comparisondict.keys())
    
    from itertools import combinations
    for combination in list(combinations(comparisondict.keys(), 2)):
        print(score, names[dfnameslist.index(combination[0])]+' vs '+ names[dfnameslist.index(combination[1])] + ' T-test')
        print(stats.ttest_ind(comparisondict[combination[0]], comparisondict[combination[1]], equal_var=False))

In [44]:
readabilityT_test = t_test_instruction_types(one_shot_readability, few_shot_readability, fine_tune_readability, score='readability')
correctnessT_test = t_test_instruction_types(one_shot_correctness, few_shot_correctness, fine_tune_correctness, score='correctness')
repitionT_test = t_test_instruction_types(one_shot_repetition, few_shot_repetition, fine_tune_repetition, score='repitition')

readability oneshot vs fewshot T-test
Ttest_indResult(statistic=-2.917103088821242, pvalue=0.004504457310469826)
readability oneshot vs finetune T-test
Ttest_indResult(statistic=-4.693695377595069, pvalue=1.1476950092825834e-05)
readability fewshot vs finetune T-test
Ttest_indResult(statistic=-1.7781301323312264, pvalue=0.07892481309464276)
correctness oneshot vs fewshot T-test
Ttest_indResult(statistic=-3.3954423397074436, pvalue=0.001107057756124154)
correctness oneshot vs finetune T-test
Ttest_indResult(statistic=-0.18827792669488633, pvalue=0.8510847774402545)
correctness fewshot vs finetune T-test
Ttest_indResult(statistic=2.9955100997537674, pvalue=0.0037773249847561875)
repitition oneshot vs fewshot T-test
Ttest_indResult(statistic=-1.5243268922307136, pvalue=0.13111331430051582)
repitition oneshot vs finetune T-test
Ttest_indResult(statistic=1.7186404438932603, pvalue=0.0894272874143118)
repitition fewshot vs finetune T-test
Ttest_indResult(statistic=3.1440978157819672, pvalue=

In [45]:
print('Readability')
mean_instruction_types(one_shot_readability, few_shot_readability, fine_tune_readability)
print('Correctness')
mean_instruction_types(one_shot_correctness, few_shot_correctness, fine_tune_correctness)
print('Repetition')
mean_instruction_types(one_shot_repetition, few_shot_repetition, fine_tune_repetition)

Readability
{'oneshot': 3.869565217391304, 'fewshot': 4.934782608695652, 'finetune': 5.456521739130435}
{'oneshot': 1.9276778851731187, 'fewshot': 1.5549515735659634, 'finetune': 1.241979096963667}
Correctness
{'oneshot': 4.543478260869565, 'fewshot': 5.478260869565218, 'finetune': 4.608695652173913}
{'oneshot': 1.601478785222242, 'fewshot': 0.9600724610334277, 'finetune': 1.718891902354734}
Repetition
{'oneshot': 2.6739130434782608, 'fewshot': 3.1739130434782608, 'finetune': 2.239130434782609}
{'oneshot': 1.3833027759851815, 'fewshot': 1.7423400334809882, 'finetune': 1.0151033359262966}


In [46]:
High_NFT_touch_1 = ['Q146', 'Q27', 'Q34', 'Q152', 'Q82', 'Q74']
High_NFT_touch_2 =  ['Q147', 'Q28', 'Q35', 'Q153', 'Q83', 'Q75']
Low_NFT_touch_1 = ['Q114', 'Q122', 'Q130', 'Q50', 'Q58', 'Q66']
Low_NFT_touch_2 =  ['Q115', 'Q123', 'Q131', 'Q51', 'Q59', 'Q69']
HighNFTtouch1df = df[High_NFT_touch_1][1:]
LowNFTtouch1df = df[Low_NFT_touch_1][1:]
HighNFTtouch2df = df[High_NFT_touch_2][1:]
LowNFTtouch2df = df[Low_NFT_touch_2][1:]

In [47]:
def t_test_haptic(df1, df2):
    dflist = [df1, df2]
    comparisondict = {'Haptic':dict(), 'NonHaptic':dict()}
    for dfname, df in list(zip(comparisondict.keys(), dflist)):
        for column in df.columns:
            comparisondict[dfname][column] = [int(a) for b in [elem for elem in df[column].tolist() if type(elem) != float] for a in b]
    
    
    for key in comparisondict.keys():
        subdict = comparisondict[key]
        comparisondict[key] = sum(subdict.values(), [])
    
    print('T-test Haptic vs Non Haptic')
    print(stats.ttest_ind(comparisondict['Haptic'], comparisondict['NonHaptic'], equal_var=True))

In [48]:
means_haptic(HighNFTtouch1df, LowNFTtouch1df)
means_haptic(HighNFTtouch2df, LowNFTtouch2df)
t_test_haptic(HighNFTtouch1df, LowNFTtouch1df)
t_test_haptic(HighNFTtouch2df, LowNFTtouch2df)

{'Haptic': 4.753623188405797, 'NonHaptic': 3.608695652173913}
{'Haptic': 1.7439458058013264, 'NonHaptic': 1.6289421983060033}
{'Haptic': 4.36231884057971, 'NonHaptic': 3.5072463768115942}
{'Haptic': 1.7147372847375846, 'NonHaptic': 1.7288485491947685}
T-test Haptic vs Non Haptic
Ttest_indResult(statistic=3.9853211202032024, pvalue=0.0001093990649665336)
T-test Haptic vs Non Haptic
Ttest_indResult(statistic=2.9169427482220938, pvalue=0.0041365783190708055)


### T-test haptic differences between finetune generated product descriptions 

In [49]:
FineTuneHH_Touch1List = ['Q34', 'Q82']
FineTuneHH_Touch2List = ['Q35', 'Q83']

FineTuneHL_Touch1List = ['Q130', 'Q66']
FineTuneHL_Touch2List = ['Q131', 'Q67']

FineTuneHH_Touch1df = df[FineTuneHH_Touch1List][1:]
FineTuneHL_Touch1df = df[FineTuneHL_Touch1List][1:]
FineTuneHH_Touch2df = df[FineTuneHH_Touch2List][1:]
FineTuneHL_Touch2df = df[FineTuneHL_Touch2List][1:]

print('haptic Criteria 1')
means_haptic(FineTuneHH_Touch1df, FineTuneHL_Touch1df)
t_test_haptic(FineTuneHH_Touch1df, FineTuneHL_Touch1df)
print('haptic Criteria 2')
means_haptic(FineTuneHH_Touch2df, FineTuneHL_Touch2df)
t_test_haptic(FineTuneHH_Touch2df, FineTuneHL_Touch2df)

haptic Criteria 1
{'Haptic': 5.695652173913044, 'NonHaptic': 3.5217391304347827}
{'Haptic': 1.184551415235307, 'NonHaptic': 1.6478516376673131}
T-test Haptic vs Non Haptic
Ttest_indResult(statistic=5.137271133831192, pvalue=6.119471993352423e-06)
haptic Criteria 2
{'Haptic': 5.130434782608695, 'NonHaptic': 3.260869565217391}
{'Haptic': 1.4864340436117256, 'NonHaptic': 1.629762707333015}
T-test Haptic vs Non Haptic
Ttest_indResult(statistic=4.064768082109021, pvalue=0.0001954902596111671)


## T-test based on the combination both criteria 

In [50]:
def t_test_haptic_combined(df1, df2, df3, df4):
    dflist = [df1, df2, df3, df4]
    comparisondict = {'Haptic1':dict(), 'Haptic2':dict(), 'NonHaptic1':dict(), 'NonHaptic2':dict()}
    for dfname, df in list(zip(comparisondict.keys(), dflist)):
        for column in df.columns:
            comparisondict[dfname][column] = [int(a) for b in [elem for elem in df[column].tolist() if type(elem) != float] for a in b]
            
    for key in comparisondict.keys():
        subdict = comparisondict[key]
        comparisondict[key] = sum(subdict.values(), [])
        
    Haptic = comparisondict['Haptic1'] + comparisondict['Haptic2']
    NonHaptic = comparisondict['NonHaptic1'] + comparisondict['NonHaptic2']
    print('T-test Haptic vs Non Haptic')
    print(stats.ttest_ind(Haptic, NonHaptic, equal_var=False))
    
t_test_haptic_combined(HighNFTtouch1df, LowNFTtouch1df, HighNFTtouch2df, LowNFTtouch2df)

T-test Haptic vs Non Haptic
Ttest_indResult(statistic=1.1545887582833112, pvalue=0.2492648694346185)


In [51]:
import statistics

one_shot_readability_list = []
for column in one_shot_readability.columns:
    one_shot_readability_list.append([elem for elem in one_shot_readability[column].tolist() if type(elem) != float])

one_shot_readability_list = [int(a) for b in one_shot_readability_list for a in b]
statistics.mean(one_shot_readability_list) 

3.869565217391304

In [52]:
def mean_instruction_types(df1, df2, df3):
    names =['oneshot', 'fewshot', 'finetune']
    dflist = [df1, df2, df3]

    comparisondict = {'df1':dict(), 'df2':dict(), 'df3':dict()}
    for dfname, df in list(zip(comparisondict.keys(), dflist)):
        for column in df.columns:
            comparisondict[dfname][column] = [int(a) for b in [elem for elem in df[column].tolist() if type(elem) != float] for a in b]
    
    for key in comparisondict.keys():
        subdict = comparisondict[key]
        comparisondict[key] = sum(subdict.values(), [])
    
    score_dict = {}
    SD_dict = {}
    dfnameslist = list(comparisondict.keys())
    
    for key, alist in list(zip(names, comparisondict.values())):
        score_dict[key] = statistics.mean(alist)
        SD_dict[key] = statistics.stdev(alist)
        
    print(score_dict)
    print(SD_dict)
    #names[dfnameslist.index(combination[0])]+' vs '+ names[dfnameslist.index(combination[1])]

mean_instruction_types(one_shot_readability, few_shot_readability, fine_tune_readability)

def means_haptic(df1, df2):
    dflist = [df1, df2]
    comparisondict = {'Haptic':dict(), 'NonHaptic':dict()}
    for dfname, df in list(zip(comparisondict.keys(), dflist)):
        for column in df.columns:
            comparisondict[dfname][column] = [int(a) for b in [elem for elem in df[column].tolist() if type(elem) != float] for a in b]
    
    for key in comparisondict.keys():
        subdict = comparisondict[key]
        comparisondict[key] = sum(subdict.values(), [])
    
    score_dict = {}
    SD_dict = {}
    dfnameslist = list(comparisondict.keys())
    for key, alist in list(zip(dfnameslist, comparisondict.values())):
        score_dict[key] = statistics.mean(alist)
        SD_dict[key] = statistics.stdev(alist)
    
    print(score_dict)
    print(SD_dict)

{'oneshot': 3.869565217391304, 'fewshot': 4.934782608695652, 'finetune': 5.456521739130435}
{'oneshot': 1.9276778851731187, 'fewshot': 1.5549515735659634, 'finetune': 1.241979096963667}


## Anova Table

In [53]:
from scipy.stats import f_oneway

In [54]:
one_shot_readability = df[['Q109', 'Q143', 'Q45', 'Q149']].loc[2:]
few_shot_readability = df[['Q117', 'Q22', 'Q53', 'Q77']].loc[2:]
fine_tune_readability = df[['Q125', 'Q29', 'Q61', 'Q69']].loc[2:]
###
one_shot_correctness = df[['Q111', 'Q144', 'Q47', 'Q150']].loc[2:]
few_shot_correctness = df[['Q119', 'Q24', 'Q55', 'Q79']].loc[2:]
fine_tune_correctness = df[['Q127', 'Q31', 'Q63', 'Q71']].loc[2:]
###
one_shot_repetition = df[['Q113', 'Q145', 'Q49', 'Q151']].loc[2:]
few_shot_repetition = df[['Q121', 'Q26', 'Q57', 'Q81']].loc[2:]
fine_tune_repetition = df[['Q129', 'Q33', 'Q65', 'Q73']].loc[2:]

In [55]:

def threeGroupAnova(df1, df2, df3):
    names =['oneshot', 'fewshot', 'finetune']
    dflist = [df1, df2, df3]

    comparisondict = {'df1':dict(), 'df2':dict(), 'df3':dict()}
    for dfname, df in list(zip(comparisondict.keys(), dflist)):
        for column in df.columns:
            comparisondict[dfname][column] = [int(a) for b in [elem for elem in df[column].tolist() if type(elem) != float] for a in b]
    
    for key in comparisondict.keys():
        subdict = comparisondict[key]
        comparisondict[key] = sum(subdict.values(), [])

    a = f_oneway(comparisondict['df1'], comparisondict['df2'], comparisondict['df3'])
    print(a)
    

In [56]:
threeGroupAnova(one_shot_readability, few_shot_readability, fine_tune_readability)
threeGroupAnova(one_shot_correctness, few_shot_correctness, fine_tune_correctness)
threeGroupAnova(one_shot_repetition, few_shot_repetition, fine_tune_repetition)

F_onewayResult(statistic=11.76117054751416, pvalue=1.9562400414427906e-05)
F_onewayResult(statistic=5.8355208880222005, pvalue=0.003709231556241496)
F_onewayResult(statistic=5.049684924866699, pvalue=0.007675494146226035)


In [57]:
from statsmodels.stats.multicomp import pairwise_tukeyhsd
def tukey_test(df1, df2, df3):
    def flat_df(df1, df2, df3):
        df_a = pd.DataFrame(columns=['val', 'group'])
        df_a['val'] = pd.Series([val for sublist in df1.values.tolist() for val in sublist if type(val) != float])
        df_a['group'] = 'one-shot'
        df_b = pd.DataFrame(columns=['val', 'group'])
        df_b['val'] = pd.Series([val for sublist in df2.values.tolist() for val in sublist if type(val) != float])
        df_b['group'] = 'few-shot'
        df_c = pd.DataFrame(columns=['val', 'group'])
        df_c['val'] = pd.Series([val for sublist in df3.values.tolist() for val in sublist if type(val) != float])
        df_c['group'] = 'fine-tune'
        return pd.concat([df_a, df_b, df_c]).reset_index(drop=True)
    
    tukeydf = flat_df(df1, df2, df3)
    tukeydf['val'] = pd.to_numeric(b.val, errors='coerce')
    tukey = pairwise_tukeyhsd(endog=b['val'], groups=b['group'], alpha=0.05)
    print(tukey)

In [58]:
tukey_test(one_shot_readability, few_shot_readability, fine_tune_readability)
tukey_test(one_shot_correctness, few_shot_correctness, fine_tune_correctness)
tukey_test(one_shot_repetition, few_shot_repetition, fine_tune_repetition)

NameError: name 'b' is not defined

### Split on haptic and gender

In [61]:
Low_haptic = ['Q109', 'Q111', 'Q113', 'Q114', 'Q115', 'Q117', 'Q119', 'Q121', 'Q122', 'Q123', 'Q125', 'Q127', 'Q129', 'Q131', 
              'Q45', 'Q47', 'Q49', 'Q50', 'Q51', 'Q53', 'Q55', 'Q57', 'Q58', 'Q59', 'Q61', 'Q63', 'Q65', 'Q66', 'Q67']

High_haptic = ['Q143', 'Q144', 'Q145', 'Q146', 'Q147', 'Q22', 'Q24', 'Q26', 'Q27', 'Q28', 'Q29', 'Q31', 'Q33', 'Q34', 'Q35' 
              , 'Q149', 'Q150', 'Q151', 'Q152', 'Q153', 'Q77', 'Q79', 'Q81', 'Q82', 'Q83', 'Q69', 'Q71', 'Q73', 'Q74', 'Q75']

In [62]:
womans_df = df[df['Q3'] == '2']
one_shot_readabilityWoman = womans_df[['Q109', 'Q143', 'Q45', 'Q149']].loc[2:]
few_shot_readabilityWoman = womans_df[['Q117', 'Q22', 'Q53', 'Q77']].loc[2:]
fine_tune_readabilityWoman = womans_df[['Q125', 'Q29', 'Q61', 'Q69']].loc[2:]
###
one_shot_correctnessWoman = womans_df[['Q111', 'Q144', 'Q47', 'Q150']].loc[2:]
few_shot_correctnessWoman = womans_df[['Q119', 'Q24', 'Q55', 'Q79']].loc[2:]
fine_tune_correctnessWoman = womans_df[['Q127', 'Q31', 'Q63', 'Q71']].loc[2:]
###
one_shot_repetitionWoman = womans_df[['Q113', 'Q145', 'Q49', 'Q151']].loc[2:]
few_shot_repetitionWoman = womans_df[['Q121', 'Q26', 'Q57', 'Q81']].loc[2:]
fine_tune_repetitionWoman = womans_df[['Q129', 'Q33', 'Q65', 'Q73']].loc[2:]

In [63]:
men_df = df[df['Q3'] == '1']
one_shot_readabilityMen = men_df[['Q109', 'Q143', 'Q45', 'Q149']].loc[2:]
few_shot_readabilityMen = men_df[['Q117', 'Q22', 'Q53', 'Q77']].loc[2:]
fine_tune_readabilityMen = men_df[['Q125', 'Q29', 'Q61', 'Q69']].loc[2:]
###
one_shot_correctnessMen = men_df[['Q111', 'Q144', 'Q47', 'Q150']].loc[2:]
few_shot_correctnessMen = men_df[['Q119', 'Q24', 'Q55', 'Q79']].loc[2:]
fine_tune_correctnessMen = men_df[['Q127', 'Q31', 'Q63', 'Q71']].loc[2:]
###
one_shot_repetitionMen = men_df[['Q113', 'Q145', 'Q49', 'Q151']].loc[2:]
few_shot_repetitionMen = men_df[['Q121', 'Q26', 'Q57', 'Q81']].loc[2:]
fine_tune_repetitionMen = men_df[['Q129', 'Q33', 'Q65', 'Q73']].loc[2:]

In [64]:
LH_one_shot_readability = one_shot_readability[[column for column in one_shot_readability.columns if column in Low_haptic]]
LH_few_shot_readability = few_shot_readability[[column for column in few_shot_readability.columns if column in Low_haptic]]
LH_finetune_readability = fine_tune_readability[[column for column in fine_tune_readability.columns if column in Low_haptic]]
HH_one_shot_readability = one_shot_readability[[column for column in one_shot_readability.columns if column in High_haptic]]
HH_few_shot_readability = few_shot_readability[[column for column in few_shot_readability.columns if column in High_haptic]]
HH_finetune_readability = fine_tune_readability[[column for column in fine_tune_readability.columns if column in High_haptic]]
LH_one_shot_correctness = one_shot_correctness[[column for column in one_shot_correctness.columns if column in Low_haptic]]
LH_few_shot_correctness = few_shot_correctness[[column for column in few_shot_correctness.columns if column in Low_haptic]]
LH_finetune_correctness = fine_tune_correctness[[column for column in fine_tune_correctness.columns if column in Low_haptic]]
HH_one_shot_correctness = one_shot_correctness[[column for column in one_shot_correctness.columns if column in High_haptic]]
HH_few_shot_correctness = few_shot_correctness[[column for column in few_shot_correctness.columns if column in High_haptic]]
HH_finetune_correctness = fine_tune_correctness[[column for column in fine_tune_correctness.columns if column in High_haptic]]
LH_one_shot_repetition = one_shot_repetition[[column for column in one_shot_repetition.columns if column in Low_haptic]]
LH_few_shot_repetition = few_shot_repetition[[column for column in few_shot_repetition.columns if column in Low_haptic]]
LH_finetune_repetition = fine_tune_repetition[[column for column in fine_tune_repetition.columns if column in Low_haptic]]
HH_one_shot_repetition = one_shot_repetition[[column for column in one_shot_repetition.columns if column in High_haptic]]
HH_few_shot_repetition = few_shot_repetition[[column for column in few_shot_repetition.columns if column in High_haptic]]
HH_finetune_repetition = fine_tune_repetition[[column for column in fine_tune_repetition.columns if column in High_haptic]]

In [65]:
print('Readability Woman Low Haptic')
mean_instruction_types(LH_one_shot_readability, LH_few_shot_readability, LH_finetune_readability)
print('Readability Woman high Haptic')
mean_instruction_types(HH_one_shot_readability, HH_few_shot_readability, HH_finetune_readability)
print('Correctness Woman Low Haptic')
mean_instruction_types(LH_one_shot_correctness, LH_few_shot_correctness, LH_finetune_correctness)
print('Correctness Woman high Haptic')
mean_instruction_types(HH_one_shot_correctness, HH_few_shot_correctness, HH_finetune_correctness)
print('Repetition Woman Low Haptic')
mean_instruction_types(LH_one_shot_repetition, LH_few_shot_repetition, LH_finetune_repetition)
print('Repetition Woman high Haptic')
mean_instruction_types(HH_one_shot_repetition, HH_few_shot_repetition, HH_finetune_repetition)

Readability Woman Low Haptic
{'oneshot': 5, 'fewshot': 4.333333333333333, 'finetune': 5.5}
{'oneshot': 2.0, 'fewshot': 1.8618986725025255, 'finetune': 1.378404875209022}
Readability Woman high Haptic
{'oneshot': 4.833333333333333, 'fewshot': 5.166666666666667, 'finetune': 5.166666666666667}
{'oneshot': 1.9407902170679516, 'fewshot': 1.7224014243685084, 'finetune': 2.228601953392904}
Correctness Woman Low Haptic
{'oneshot': 5.5, 'fewshot': 5.166666666666667, 'finetune': 5.166666666666667}
{'oneshot': 0.5477225575051661, 'fewshot': 1.1690451944500122, 'finetune': 1.6020819787597222}
Correctness Woman high Haptic
{'oneshot': 4.5, 'fewshot': 5.833333333333333, 'finetune': 5.833333333333333}
{'oneshot': 1.6431676725154984, 'fewshot': 0.408248290463863, 'finetune': 0.752772652709081}
Repetition Woman Low Haptic
{'oneshot': 3, 'fewshot': 3, 'finetune': 2.1666666666666665}
{'oneshot': 2.0, 'fewshot': 2.0, 'finetune': 0.983192080250175}
Repetition Woman high Haptic
{'oneshot': 2.333333333333333

### split on gender

In [68]:
men_df = df[(df['Q3'] == '1') & df['Finished'] == 1]
one_shot_readabilityMen = men_df[['Q109', 'Q143', 'Q45', 'Q149']].loc[2:]
few_shot_readabilityMen = men_df[['Q117', 'Q22', 'Q53', 'Q77']].loc[2:]
fine_tune_readabilityMen = men_df[['Q125', 'Q29', 'Q61', 'Q69']].loc[2:]
###
one_shot_correctnessMen = men_df[['Q111', 'Q144', 'Q47', 'Q150']].loc[2:]
few_shot_correctnessMen = men_df[['Q119', 'Q24', 'Q55', 'Q79']].loc[2:]
fine_tune_correctnessMen = men_df[['Q127', 'Q31', 'Q63', 'Q71']].loc[2:]
###
one_shot_repetitionMen = men_df[['Q113', 'Q145', 'Q49', 'Q151']].loc[2:]
few_shot_repetitionMen = men_df[['Q121', 'Q26', 'Q57', 'Q81']].loc[2:]
fine_tune_repetitionMen = men_df[['Q129', 'Q33', 'Q65', 'Q73']].loc[2:]

In [69]:
womans_df = df[(df['Q3'] == '2') & df['Finished'] == 1]
one_shot_readabilityWoman = womans_df[['Q109', 'Q143', 'Q45', 'Q149']].loc[2:]
few_shot_readabilityWoman = womans_df[['Q117', 'Q22', 'Q53', 'Q77']].loc[2:]
fine_tune_readabilityWoman = womans_df[['Q125', 'Q29', 'Q61', 'Q69']].loc[2:]
###
one_shot_correctnessWoman = womans_df[['Q111', 'Q144', 'Q47', 'Q150']].loc[2:]
few_shot_correctnessWoman = womans_df[['Q119', 'Q24', 'Q55', 'Q79']].loc[2:]
fine_tune_correctnessWoman = womans_df[['Q127', 'Q31', 'Q63', 'Q71']].loc[2:]
###
one_shot_repetitionWoman = womans_df[['Q113', 'Q145', 'Q49', 'Q151']].loc[2:]
few_shot_repetitionWoman = womans_df[['Q121', 'Q26', 'Q57', 'Q81']].loc[2:]
fine_tune_repetitionWoman = womans_df[['Q129', 'Q33', 'Q65', 'Q73']].loc[2:]

In [71]:
print('Readability Woman')
mean_instruction_types(one_shot_readabilityWoman, few_shot_readabilityWoman, fine_tune_readabilityWoman)
print('Correctness Woman')
mean_instruction_types(one_shot_correctnessWoman, few_shot_correctnessWoman, fine_tune_correctnessWoman)
print('Repetition Woman')
mean_instruction_types(one_shot_repetitionWoman, few_shot_repetitionWoman, fine_tune_repetitionWoman)

Readability Woman
{'oneshot': 4.916666666666667, 'fewshot': 4.75, 'finetune': 5.333333333333333}
{'oneshot': 1.880924981991251, 'fewshot': 1.764549903980152, 'finetune': 1.775250729197189}
Correctness Woman
{'oneshot': 5, 'fewshot': 5.5, 'finetune': 5.5}
{'oneshot': 1.2792042981336627, 'fewshot': 0.9045340337332909, 'finetune': 1.243163121016122}
Repetition Woman
{'oneshot': 2.6666666666666665, 'fewshot': 3.6666666666666665, 'finetune': 2.25}
{'oneshot': 1.6696942198734437, 'fewshot': 1.7232808737106582, 'finetune': 1.4222261679238197}


In [72]:
print('Readability Men')
mean_instruction_types(one_shot_readabilityMen, few_shot_readabilityMen, fine_tune_readabilityMen)
print('Correctness Men')
mean_instruction_types(one_shot_correctnessMen, few_shot_correctnessMen, fine_tune_correctnessMen)
print('Repetition Men')
mean_instruction_types(one_shot_repetitionMen, few_shot_repetitionMen, fine_tune_repetitionMen)

Readability Men
{'oneshot': 3.5, 'fewshot': 5, 'finetune': 5.5}
{'oneshot': 1.8298865671086961, 'fewshot': 1.497472618255253, 'finetune': 1.02247471629109}
Correctness Men
{'oneshot': 4.382352941176471, 'fewshot': 5.470588235294118, 'finetune': 4.294117647058823}
{'oneshot': 1.6880095124382435, 'fewshot': 0.9919461775980272, 'finetune': 1.7671996639900136}
Repetition Men
{'oneshot': 2.676470588235294, 'fewshot': 3, 'finetune': 2.235294117647059}
{'oneshot': 1.2961618921353482, 'fewshot': 1.7407765595569784, 'finetune': 0.8548905132596332}


In [73]:
readabilityT_testWoman = t_test_instruction_types(one_shot_readabilityWoman, few_shot_readabilityWoman, fine_tune_readabilityWoman, score='readability')
correctnessT_testWoman = t_test_instruction_types(one_shot_correctnessWoman, few_shot_correctnessWoman, fine_tune_correctnessWoman,  score='correctness')
repitionT_testWoman = t_test_instruction_types(one_shot_repetitionWoman, few_shot_repetitionWoman, fine_tune_repetitionWoman, score='repitition')

readability oneshot vs fewshot T-test
Ttest_indResult(statistic=0.22386133032840014, pvalue=0.8249393635916926)
readability oneshot vs finetune T-test
Ttest_indResult(statistic=-0.5580665539124304, pvalue=0.5824528985888504)
readability fewshot vs finetune T-test
Ttest_indResult(statistic=-0.8073124087192437, pvalue=0.4281294258317254)
correctness oneshot vs fewshot T-test
Ttest_indResult(statistic=-1.1055415967851332, pvalue=0.2821778681279097)
correctness oneshot vs finetune T-test
Ttest_indResult(statistic=-0.9710083124552245, pvalue=0.34210813232034276)
correctness fewshot vs finetune T-test
Ttest_indResult(statistic=0.0, pvalue=1.0)
repitition oneshot vs fewshot T-test
Ttest_indResult(statistic=-1.4436795095573636, pvalue=0.16293328380686306)
repitition oneshot vs finetune T-test
Ttest_indResult(statistic=0.658081200264627, pvalue=0.5174848989067073)
repitition fewshot vs finetune T-test
Ttest_indResult(statistic=2.1963550989739966, pvalue=0.03930373951551628)


In [74]:
readabilityT_testMen = t_test_instruction_types(one_shot_readabilityMen, few_shot_readabilityMen, fine_tune_readabilityMen,  score='readability')
correctnessT_testMen = t_test_instruction_types(one_shot_correctnessMen, few_shot_correctnessMen, fine_tune_correctnessMen,  score='correctness')
repitionT_testMen = t_test_instruction_types(one_shot_repetitionMen, few_shot_repetitionMen, fine_tune_repetitionMen, score='repitition')

readability oneshot vs fewshot T-test
Ttest_indResult(statistic=-3.699044042623485, pvalue=0.0004539390899297933)
readability oneshot vs finetune T-test
Ttest_indResult(statistic=-5.563427373295263, pvalue=9.437230227395297e-07)
readability fewshot vs finetune T-test
Ttest_indResult(statistic=-1.607872338284728, pvalue=0.11326811884177118)
correctness oneshot vs fewshot T-test
Ttest_indResult(statistic=-3.2409622517294987, pvalue=0.002054346424764269)
correctness oneshot vs finetune T-test
Ttest_indResult(statistic=0.21052729757309366, pvalue=0.8339061258180107)
correctness fewshot vs finetune T-test
Ttest_indResult(statistic=3.385016001931651, pvalue=0.0013622996584491543)
repitition oneshot vs fewshot T-test
Ttest_indResult(statistic=-0.8692148938516142, pvalue=0.3881375645924976)
repitition oneshot vs finetune T-test
Ttest_indResult(statistic=1.6567790500553776, pvalue=0.10304558164562963)
repitition fewshot vs finetune T-test
Ttest_indResult(statistic=2.2991859264032164, pvalue=0.0

# Not Split

In [43]:
one_shot_readability = df[['Q109', 'Q143', 'Q45', 'Q149']].loc[2:]
few_shot_readability = df[['Q117', 'Q22', 'Q53', 'Q77']].loc[2:]
fine_tune_readability = df[['Q125', 'Q29', 'Q61', 'Q69']].loc[2:]
###
one_shot_correctness = df[['Q111', 'Q144', 'Q47', 'Q150']].loc[2:]
few_shot_correctness = df[['Q119', 'Q24', 'Q55', 'Q79']].loc[2:]
fine_tune_correctness = df[['Q127', 'Q31', 'Q63', 'Q71']].loc[2:]
###
one_shot_repetition = df[['Q113', 'Q145', 'Q49', 'Q151']].loc[2:]
few_shot_repetition = df[['Q121', 'Q26', 'Q57', 'Q81']].loc[2:]
fine_tune_repetition = df[['Q129', 'Q33', 'Q65', 'Q73']].loc[2:]

In [47]:
print('Readability')
mean_instruction_types(one_shot_readability, few_shot_readability, fine_tune_readability)
print('Correctness')
mean_instruction_types(one_shot_correctness, few_shot_correctness, fine_tune_correctness)
print('Repetition')
mean_instruction_types(one_shot_repetition, few_shot_repetition, fine_tune_repetition)

Readability
{'oneshot': 3.869565217391304, 'fewshot': 4.934782608695652, 'finetune': 5.456521739130435}
{'oneshot': 1.9276778851731187, 'fewshot': 1.5549515735659634, 'finetune': 1.241979096963667}
Correctness
{'oneshot': 4.543478260869565, 'fewshot': 5.478260869565218, 'finetune': 4.608695652173913}
{'oneshot': 1.601478785222242, 'fewshot': 0.9600724610334277, 'finetune': 1.718891902354734}
Repetition
{'oneshot': 2.6739130434782608, 'fewshot': 3.1739130434782608, 'finetune': 2.239130434782609}
{'oneshot': 1.3833027759851815, 'fewshot': 1.7423400334809882, 'finetune': 1.0151033359262966}
